## multi-frequency

In [ ]:
#!/usr/bin/env python
# coding: utf-8

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import numpy as np
from astropy.io import fits
import datetime
import os

# Set Matplotlib font
matplotlib.rc('font', family='Times New Roman', size=12)

# Create output directory if it doesn't exist
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

# Define FITS files and corresponding colors
files = {
    '3270': {'file': 'nrh2_3270_h80_20240529_141500c01_b.fts', 'color': 'brown'},
    '2987': {'file': 'nrh2_2987_h80_20240529_141500c01_b.fts', 'color': 'orange'},
    '2706': {'file': 'nrh2_2706_h80_20240529_141500c01_b.fts', 'color': 'purple'},
    '2280': {'file': 'nrh2_2280_h80_20240529_141500c01_b.fts', 'color': 'cyan'},
    '1732': {'file': 'nrh2_1732_h80_20240529_141500c01_b.fts', 'color': 'magenta'},
    '1509': {'file': 'nrh2_1509_h80_20240529_141500c01_b.fts', 'color': 'darkviolet'}
}

# Load one FITS file to get header information (assuming headers are consistent across files)
hdu = fits.open(files['1509']['file'])

# Define image extent in arcseconds
extent = (
    -hdu[1].header['CRPIX1'] / hdu[1].header['SOLAR_R'] * 960,  # xmin
    +hdu[1].header['CRPIX1'] / hdu[1].header['SOLAR_R'] * 960,  # xmax
    -hdu[1].header['CRPIX1'] / hdu[1].header['SOLAR_R'] * 960,  # ymin
    +hdu[1].header['CRPIX1'] / hdu[1].header['SOLAR_R'] * 960   # ymax
)

# Extract start time from filename
file_name = os.path.basename(files['1509']['file'])
date_time_str = file_name.split('_')[3]  # Extract '20240529'
time_str = file_name.split('_')[4][:6]   # Extract '141500'
start_time = datetime.datetime.strptime(
    f"{date_time_str} {time_str}",
    '%Y%m%d %H%M%S'
)

# Set time step to 250 milliseconds (0.25 seconds)
time_step = 0.25  # seconds
no_images = len(hdu[1].data)

# Define resume time (2024-05-29 14:20:53.500)
resume_time = datetime.datetime.strptime('20240529 142649.250', '%Y%m%d %H%M%S.%f')

# Calculate the starting image index
time_diff = (resume_time - start_time).total_seconds()
start_img_no = int(time_diff / time_step)

# Ensure start_img_no is within valid range
if start_img_no < 0 or start_img_no >= no_images:
    print(f"Error: Resume time {resume_time} is out of range for the data cube.")
    hdu.close()
    exit()

# Load data cubes for all frequencies
data_nrh = {}
for freq, info in files.items():
    hdu_freq = fits.open(info['file'])
    data_nrh[freq] = hdu_freq[1].data
    hdu_freq.close()

hdu.close()

# Loop through images starting from start_img_no
for i in range(start_img_no, no_images):
    # Create plot
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot2grid((1, 1), (0, 0))

    # Plot contours for each frequency and store for legend
    legend_handles = []
    for freq, info in files.items():
        data = data_nrh[freq][i][2]  # Stokes I data array at the specified time
        max_NRH = np.max(data)
        level = [0.5 * max_NRH, 0.95 * max_NRH]
        plt.contour(data, levels=level, colors=info['color'], extent=extent)
        # Create a custom legend handle for this frequency
        legend_handles.append(mlines.Line2D([], [], color=info['color'], label=f'{freq[:-1]}.{freq[-1]} MHz'))

    # Add a circle representing one solar diameter
    kk = hdu[1].header['SOLAR_R'] * 960 / 60
    circle = plt.Circle((0, 0), kk, color='red', fill=False)
    ax.add_artist(circle)

    # Get the date and time of the image
    img_time = start_time + datetime.timedelta(seconds=i * time_step)
    img_time_str = img_time.strftime('%Y%m%d_%H%M%S%f')[:-3]  # Format for filename (e.g., 20240529_141500250)

    # Set title with date and time
    plt.title(f"Image Time: {img_time.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]}", pad=20)

    # Add legend with custom handles
    plt.legend(handles=legend_handles, loc='upper right')

    # Set axis labels and aspect ratio
    plt.xlabel('X (arcseconds)')
    plt.ylabel('Y (arcseconds)')
    ax.set_aspect('equal')  # Ensure square plot

    # Save plot with timestamp in filename to output directory
    output_path = os.path.join(output_dir, f'nrh_multi_freq_{img_time_str}_contour.png')
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close(fig)  # Close figure to free memory
    print(f"Saved image: {output_path}")

## specific time

In [ ]:
#!/usr/bin/env python
# coding: utf-8

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import numpy as np
from astropy.io import fits
import datetime
import os

# Set Matplotlib font
matplotlib.rc('font', family='Times New Roman', size=12)

# Create output directory if it doesn't exist
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

# Define FITS files and corresponding colors
files = {
    '1509': {'file': 'nrh2_1509_h80_20240529_141500c01_b.fts', 'color': 'darkgreen'},
    '1732': {'file': 'nrh2_1732_h80_20240529_141500c01_b.fts', 'color': 'brown'},
    '2280': {'file': 'nrh2_2280_h80_20240529_141500c01_b.fts', 'color': 'blue'},
    '2706': {'file': 'nrh2_2706_h80_20240529_141500c01_b.fts', 'color': 'purple'}
}

# Load one FITS file to get header information (assuming headers are consistent across files)
hdu = fits.open(files['1509']['file'])

# Define image extent in arcseconds
extent = (
    -hdu[1].header['CRPIX1'] / hdu[1].header['SOLAR_R'] * 960,  # xmin
    +hdu[1].header['CRPIX1'] / hdu[1].header['SOLAR_R'] * 960,  # xmax
    -hdu[1].header['CRPIX1'] / hdu[1].header['SOLAR_R'] * 960,  # ymin
    +hdu[1].header['CRPIX1'] / hdu[1].header['SOLAR_R'] * 960   # ymax
)

# Extract start time from filename
file_name = os.path.basename(files['1509']['file'])
date_time_str = file_name.split('_')[3]  # Extract '20240529'
time_str = file_name.split('_')[4][:6]   # Extract '141500'
start_time = datetime.datetime.strptime(
    f"{date_time_str} {time_str}",
    '%Y%m%d %H%M%S'
)

# Set time step to 250 milliseconds (0.25 seconds)
time_step = 0.25  # seconds
no_images = len(hdu[1].data)

# Define four specific times to plot (replace with your desired times)
plot_times = [
    datetime.datetime.strptime('20240529 142652.000', '%Y%m%d %H%M%S.%f'),
    datetime.datetime.strptime('20240529 142815.000', '%Y%m%d %H%M%S.%f'),
    datetime.datetime.strptime('20240529 142940.000', '%Y%m%d %H%M%S.%f'),
    datetime.datetime.strptime('20240529 143036.000', '%Y%m%d %H%M%S.%f')
]

# Load data cubes for all frequencies
data_nrh = {}
for freq, info in files.items():
    hdu_freq = fits.open(info['file'])
    data_nrh[freq] = hdu_freq[1].data
    hdu_freq.close()

hdu.close()

# Plot for each specified time
for plot_time in plot_times:
    # Calculate image index for the current time
    time_diff = (plot_time - start_time).total_seconds()
    img_no = int(time_diff / time_step)  # Image index based on 0.25s intervals

    # Check if img_no is within valid range
    if img_no < 0 or img_no >= no_images:
        print(f"Error: Time {plot_time} is out of range for the data cube.")
        continue

    # Create plot
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot2grid((1, 1), (0, 0))

    # Plot contours for each frequency and store for legend
    legend_handles = []
    for freq, info in files.items():
        data = data_nrh[freq][img_no][2]  # Stokes I data array at the specified time
        max_NRH = np.max(data)
        level = [0.5 * max_NRH, 0.95 * max_NRH]
        plt.contour(data, levels=level, colors=info['color'], extent=extent)
        # Create a custom legend handle for this frequency
        legend_handles.append(mlines.Line2D([], [], color=info['color'], label=f'{freq[:-1]}.{freq[-1]} MHz'))

    # Add a single red solar disk
    kk = 960
    circle = plt.Circle((0, 0), kk, color='red', fill=False)
    ax.add_artist(circle)

    # Set title with date and time
    img_time_str = plot_time.strftime('%Y%m%d_%H%M%S_%f')[:-3]
    plt.title(f"Image Time: {plot_time.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]}")

    # Add legend with custom handles
    plt.legend(handles=legend_handles, loc='upper right')

    # Set axis labels and aspect ratio
    plt.xlabel('X (arcseconds)')
    plt.ylabel('Y (arcseconds)')
    ax.set_aspect('equal')  # Ensure square plot

    # Save plot with timestamp in filename to output directory
    output_path = os.path.join(output_dir, f'nrh_multi_freq_{img_time_str}_contour.png')
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close(fig)  # Close figure to free memory
    print(f"Saved image: {output_path}")